In [3]:
!pip install openpyxl
!pip install psycopg2-binary


  Using cached psycopg2_binary-2.9.9-cp39-cp39-macosx_10_9_x86_64.whl (2.8 MB)


# ETL: JSON TO EXCEL: HISTORIC_WEATHER

In [ ]:
import json
import pandas as pd
from datetime import datetime

# Load the JSON file
json_file_path = '/Users/ardisopi/Documents/DWH/historic_weather.json'
with open(json_file_path, 'r') as file:
    data = json.load(file)

# Initialize an empty list to store the extracted information
extracted_data = []

# Function to safely extract data with default fallback
def safe_get(d, keys, default=None):
    for key in keys:
        try:
            d = d[key]
        except (KeyError, IndexError, TypeError):
            return default
    return d

# Function to convert Kelvin to Celsius
def kelvin_to_celsius(kelvin):
    return kelvin - 273.15

# Loop through each city's weather data
for city_data in data:
    city_id = city_data['city_id']
    weather_list = city_data['list']
    
    for weather in weather_list:
        dt = safe_get(weather, ['dt'])
        main = safe_get(weather, ['main'], {})
        wind = safe_get(weather, ['wind'], {})
        clouds = safe_get(weather, ['clouds', 'all'], None)
        weather_description = safe_get(weather, ['weather', 0, 'description'], None)
        
        # Convert temperatures from Kelvin to Celsius
        temp_celsius = kelvin_to_celsius(main.get('temp')) if main.get('temp') is not None else None
        
        
        # Convert Unix timestamp to human-readable format
        if dt is not None:
            dt_obj = datetime.utcfromtimestamp(dt)
            date_str = dt_obj.date().isoformat()
            time_str = dt_obj.time().isoformat()
            datetime_str = dt_obj.isoformat()
        else:
            date_str = time_str = datetime_str = None
        
        extracted_data.append({
            'city_id': city_id,
            'datetime': datetime_str,
            'date': date_str,
            'time': time_str,
            'temp': temp_celsius,
            'pressure': main.get('pressure'),
            'humidity': main.get('humidity'),
            'wind_speed': wind.get('speed'),
            'clouds': clouds,
            'weather_description': weather_description
        })

# Create a DataFrame from the extracted data
extracted_df = pd.DataFrame(extracted_data)

# Display the DataFrame
print(extracted_df.head())

# Optionally, save the extracted data to a new Excel file
output_path = '/Users/ardisopi/Documents/DWH/extracted_weather_data.xlsx'
extracted_df.to_excel(output_path, index=False)
print(f"Extracted data saved to {output_path}")


# ETL: JSON TO EXCEL: FIXTURES.JSON

In [ ]:
import json
import pandas as pd
from datetime import datetime

# Load the JSON file
json_file_path = '/mnt/data/2024-05-18+20_24+-+fixtures.json'
with open(json_file_path, 'r') as file:
    data = json.load(file)

# Initialize an empty list to store the extracted information
extracted_data = []

# Function to safely extract data with default fallback
def safe_get(d, keys, default=None):
    for key in keys:
        try:
            d = d[key]
        except (KeyError, IndexError, TypeError):
            return default
    return d

# Function to split date and time
def split_date_time(datetime_str):
    dt_obj = datetime.fromisoformat(datetime_str.replace('Z', '+00:00'))
    date_str = dt_obj.date().isoformat()
    time_str = dt_obj.time().isoformat()
    return date_str, time_str

# Loop through each fixture data
for fixture_data in data:
    fixture = fixture_data['fixture']
    venue = fixture['venue']
    teams = fixture_data['teams']
    goals = fixture_data['goals']
    
    fixture_id = fixture['id']
    referee = fixture['referee']
    datetime_str = fixture['date']
    timestamp = fixture['timestamp']
    status_short = fixture['status']['short']
    venue_id = venue['id']
    venue_name = venue['name']
    venue_city = venue['city']
    
    home_team_id = teams['home']['id']
    home_team_name = teams['home']['name']
    home_winner = teams['home']['winner']
    home_goals = goals['home']
    
    away_team_id = teams['away']['id']
    away_team_name = teams['away']['name']
    away_winner = teams['away']['winner']
    away_goals = goals['away']
    
    date, time = split_date_time(datetime_str)
    
    extracted_data.append({
        'fixture_id': fixture_id,
        'referee': referee,
        'date': date,
        'time': time,
        'timestamp': timestamp,
        'status_short': status_short,
        'venue_id': venue_id,
        'venue_name': venue_name,
        'venue_city': venue_city,
        'home_team_id': home_team_id,
        'home_team_name': home_team_name,
        'home_winner': home_winner,
        'home_goals': home_goals,
        'away_team_id': away_team_id,
        'away_team_name': away_team_name,
        'away_winner': away_winner,
        'away_goals': away_goals
    })

# Create a DataFrame from the extracted data
extracted_df = pd.DataFrame(extracted_data)

# Save the extracted data to a new Excel file
output_path = '/mnt/data/extracted_fixtures_data_updated.xlsx'
extracted_df.to_excel(output_path, index=False)

# Display the path to the output file
output_path

# ETL: JSON TO EXCEL: PREDICTIONS4DAYS.JSON

In [43]:
import json
import pandas as pd
from datetime import datetime

# Load the JSON file
json_file_path = '/Users/ardisopi/Documents/DWH/JSON/2024-05-20+16_42+-+predictions4days.json'
with open(json_file_path, 'r') as file:
    data = json.load(file)

# Initialize an empty list to store the extracted information
extracted_data = []

# Function to safely extract data with default fallback
def safe_get(d, keys, default=None):
    for key in keys:
        try:
            d = d[key]
        except (KeyError, IndexError, TypeError):
            return default
    return d

# Function to convert Kelvin to Celsius
def kelvin_to_celsius(kelvin):
    return kelvin - 273.15

# Loop through each city's weather data
for city_data in data:
    city_info = city_data['city']
    city_id = city_info['id']
    city_name = city_info['name']
    coord_lon = city_info['coord']['lon']
    coord_lat = city_info['coord']['lat']
    country = city_info['country']
    population = city_info['population']
    timezone = city_info['timezone']
    
    weather_list = city_data['list']
    
    for weather in weather_list:
        dt = safe_get(weather, ['dt'])
        temp_day_kelvin = safe_get(weather, ['main', 'temp'])
        pressure = safe_get(weather, ['main', 'pressure'])
        humidity = safe_get(weather, ['main', 'humidity'])
        weather_description = safe_get(weather, ['weather', 0, 'description'])
        wind_speed = safe_get(weather, ['wind', 'speed'])
        wind_deg = safe_get(weather, ['wind', 'deg'])
        rain = safe_get(weather, ['rain', '1h'], 0)

        # Convert temperatures from Kelvin to Celsius
        temp_day_celsius = kelvin_to_celsius(temp_day_kelvin) if temp_day_kelvin is not None else None
        
        # Convert Unix timestamp to human-readable format
        if dt is not None:
            dt_obj = datetime.utcfromtimestamp(dt)
            date_str = dt_obj.date().isoformat()
            time_str = dt_obj.time().isoformat()
        else:
            date_str = time_str = None
        
        extracted_data.append({
            'city_id': city_id,
            'city_name': city_name,
            'coord_lon': coord_lon,
            'coord_lat': coord_lat,
            'country': country,
            'population': population,
            'timezone': timezone,
            'date': date_str,
            'time': time_str,
            'temp_day': temp_day_celsius,
            'pressure': pressure,
            'humidity': humidity,
            'weather_description': weather_description,
            'wind_speed': wind_speed,
            'wind_deg': wind_deg,
            'rain': rain
        })

# Create a DataFrame from the extracted data
extracted_df = pd.DataFrame(extracted_data)

# Save the extracted data to a new Excel file
output_path = '/Users/ardisopi/Documents/DWH/extracted_weather_data_4days.xlsx'
extracted_df.to_excel(output_path, index=False)

# Display the path to the output file
output_path

'/Users/ardisopi/Documents/DWH/extracted_weather_data_4days.xlsx'

# JSON Exploration - STATISTICS_FIXTURES_ALL.JSON

In [22]:
import json

# Load the JSON file
file_path = '/Users/ardisopi/Documents/DWH/JSON/2024-05-18+20_24+-+statistics_fixtures_all.json'
with open(file_path, 'r') as file:
    raw_data = file.read()

# Parse the entire JSON content
data = json.loads(raw_data)

# Print the structure of the first few entries to understand the data
for i, entry in enumerate(data[:5]):
    print(f"Entry {i}: {json.dumps(entry, indent=2)}\n")


Entry 0: "{\"get\":\"fixtures\\/statistics\",\"parameters\":{\"fixture\":\"1037952\"},\"errors\":[],\"results\":2,\"paging\":{\"current\":1,\"total\":1},\"response\":[{\"team\":{\"id\":723,\"name\":\"Almeria\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/723.png\"},\"statistics\":[{\"type\":\"Shots on Goal\",\"value\":5},{\"type\":\"Shots off Goal\",\"value\":11},{\"type\":\"Total Shots\",\"value\":19},{\"type\":\"Blocked Shots\",\"value\":3},{\"type\":\"Shots insidebox\",\"value\":12},{\"type\":\"Shots outsidebox\",\"value\":7},{\"type\":\"Fouls\",\"value\":16},{\"type\":\"Corner Kicks\",\"value\":8},{\"type\":\"Offsides\",\"value\":1},{\"type\":\"Ball Possession\",\"value\":\"60%\"},{\"type\":\"Yellow Cards\",\"value\":1},{\"type\":\"Red Cards\",\"value\":null},{\"type\":\"Goalkeeper Saves\",\"value\":2},{\"type\":\"Total passes\",\"value\":480},{\"type\":\"Passes accurate\",\"value\":422},{\"type\":\"Passes %\",\"value\":\"88%\"},{\"type\":\"expected_goals\",\"val

# ETL: JSON TO DF - STATISTICS_FIXTURES_ALL

In [33]:
import json
import pandas as pd

# Load the JSON file
file_path = '/Users/ardisopi/Documents/DWH/JSON/2024-05-18+20_24+-+statistics_fixtures_all.json'
with open(file_path, 'r') as file:
    raw_data = file.readlines()

# Initialize an empty list to store the extracted information
extracted_data = []

# Loop through each line and parse the JSON string
for line in raw_data:
    try:
        # First parse the line to get the list of JSON strings
        outer_list = json.loads(line)
        for json_str in outer_list:
            # Then parse each JSON string in the list
            fixture = json.loads(json_str)
            fixture_id = fixture['parameters']['fixture']
            for fixture_data in fixture['response']:
                team = fixture_data['team']
                statistics = fixture_data['statistics']
                
                team_id = team['id']
                team_name = team['name']
                
                stats_dict = {stat['type']: stat['value'] for stat in statistics}
                
                extracted_data.append({
                    'fixtureID': fixture_id,
                    'team_Id': team_id,
                    'team_name': team_name,
                    'Shots on Goal': stats_dict.get('Shots on Goal'),
                    'Shots off Goal': stats_dict.get('Shots off Goal'),
                    'Total Shots': stats_dict.get('Total Shots'),
                    'Blocked Shots': stats_dict.get('Blocked Shots'),
                    'Shots insidebox': stats_dict.get('Shots insidebox'),
                    'Shots outsidebox': stats_dict.get('Shots outsidebox'),
                    'Fouls': stats_dict.get('Fouls'),
                    'Corner Kicks': stats_dict.get('Corner Kicks'),
                    'Offsides': stats_dict.get('Offsides'),
                    'Ball Possession': stats_dict.get('Ball Possession'),
                    'Yellow Cards': stats_dict.get('Yellow Cards'),
                    'Red Cards': stats_dict.get('Red Cards'),
                    'Goalkeeper Saves': stats_dict.get('Goalkeeper Saves'),
                    'Total passes': stats_dict.get('Total passes'),
                    'Passes accurate': stats_dict.get('Passes accurate'),
                    'Passes %': stats_dict.get('Passes %'),
                    'expected_goals': stats_dict.get('expected_goals')
                })
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        continue

# Create a DataFrame from the extracted data
extracted_df = pd.DataFrame(extracted_data)

# Display the DataFrame
print(extracted_df.head())


  fixtureID  team_Id        team_name  Shots on Goal  Shots off Goal  \
0   1037952      723          Almeria              5              11   
1   1037952      728   Rayo Vallecano              4               2   
2   1037953      531    Athletic Club              1               3   
3   1037953      541      Real Madrid              8               3   
4   1037954      530  Atletico Madrid              6               6   

   Total Shots  Blocked Shots  Shots insidebox  Shots outsidebox  Fouls  \
0           19              3             12.0               7.0     16   
1           10              4              6.0               4.0     25   
2            7              3              5.0               2.0     18   
3           14              3             10.0               4.0      6   
4           14              2             11.0               3.0     15   

   Corner Kicks  Offsides Ball Possession  Yellow Cards  Red Cards  \
0           8.0       1.0             60%     


# JSON EXPLORATION - STATISTICS_PLAYERS_ALL.JSON


In [35]:
import json

# Load the JSON file
file_path = '/Users/ardisopi/Documents/DWH/JSON/2024-05-18+20_24+-+statistics_players_all.json'
with open(file_path, 'r') as file:
    raw_data = file.read()

# Parse the entire JSON content
data = json.loads(raw_data)

# Print the structure of the first few entries to understand the data
for i, entry in enumerate(data[:1]):
    print(f"Entry {i}: {json.dumps(entry, indent=2)}\n")

Entry 0: "{\"get\":\"fixtures\\/players\",\"parameters\":{\"fixture\":\"1037952\"},\"errors\":[],\"results\":2,\"paging\":{\"current\":1,\"total\":1},\"response\":[{\"team\":{\"id\":723,\"name\":\"Almeria\",\"logo\":\"https:\\/\\/media.api-sports.io\\/football\\/teams\\/723.png\",\"update\":\"2024-05-18T04:06:02+00:00\"},\"players\":[{\"player\":{\"id\":46870,\"name\":\"Diego Mari\\u00f1o\",\"photo\":\"https:\\/\\/media.api-sports.io\\/football\\/players\\/46870.png\"},\"statistics\":[{\"games\":{\"minutes\":90,\"number\":1,\"position\":\"G\",\"rating\":\"6.9\",\"captain\":false,\"substitute\":false},\"offsides\":null,\"shots\":{\"total\":null,\"on\":null},\"goals\":{\"total\":null,\"conceded\":2,\"assists\":null,\"saves\":2},\"passes\":{\"total\":21,\"key\":null,\"accuracy\":\"17\"},\"tackles\":{\"total\":null,\"blocks\":null,\"interceptions\":null},\"duels\":{\"total\":null,\"won\":null},\"dribbles\":{\"attempts\":null,\"success\":null,\"past\":null},\"fouls\":{\"drawn\":null,\"commi

# ETL: JSON TO DF - STATISTICS_PLAYERS_ALL.JSON

In [85]:
import json
import pandas as pd

# Load the JSON file
file_path = '/Users/ardisopi/Documents/DWH/JSON/2024-05-18+20_24+-+statistics_players_all.json'
with open(file_path, 'r') as file:
    raw_data = file.readlines()

# Initialize an empty list to store the extracted information
extracted_data = []

# Loop through each line and parse the JSON string
for line in raw_data:
    try:
        # First parse the line to get the list of JSON strings
        outer_list = json.loads(line)
        for json_str in outer_list:
            # Then parse each JSON string in the list
            fixture = json.loads(json_str)
            fixture_id = fixture['parameters']['fixture']
            for team_data in fixture['response']:
                team = team_data['team']
                team_id = team['id']
                team_name = team['name']
                
                for player_data in team_data['players']:
                    player = player_data['player']
                    statistics = player_data['statistics'][0]  # assuming there's always one statistics entry per player

                    player_id = player['id']
                    player_name = player['name']
                    
                    extracted_data.append({
                        'fixtureID': fixture_id,
                        'team_Id': team_id,
                        'team_name': team_name,
                        'player_id': player_id,
                        'player_name': player_name,
                        'minutes': statistics['games'].get('minutes'),
                        'number': statistics['games'].get('number'),
                        'position': statistics['games'].get('position'),
                        'rating': statistics['games'].get('rating'),
                        'captain': statistics['games'].get('captain'),
                        'substitute': statistics['games'].get('substitute'),
                        'offsides': statistics.get('offsides'),
                        'shots_total': statistics['shots'].get('total'),
                        'shots_on': statistics['shots'].get('on'),
                        'goals_total': statistics['goals'].get('total'),
                        'goals_conceded': statistics['goals'].get('conceded'),
                        'goals_assists': statistics['goals'].get('assists'),
                        'saves': statistics['goals'].get('saves'),
                        'passes_total': statistics['passes'].get('total'),
                        'passes_key': statistics['passes'].get('key'),
                        'passes_accuracy': statistics['passes'].get('accuracy'),
                        'tackles_total': statistics['tackles'].get('total'),
                        'tackles_blocks': statistics['tackles'].get('blocks'),
                        'tackles_interceptions': statistics['tackles'].get('interceptions'),
                        'duels_total': statistics['duels'].get('total'),
                        'duels_won': statistics['duels'].get('won'),
                        'dribbles_attempts': statistics['dribbles'].get('attempts'),
                        'dribbles_success': statistics['dribbles'].get('success'),
                        'dribbles_past': statistics['dribbles'].get('past'),
                        'fouls_drawn': statistics['fouls'].get('drawn'),
                        'fouls_committed': statistics['fouls'].get('committed'),
                        'yellow_cards': statistics['cards'].get('yellow'),
                        'red_cards': statistics['cards'].get('red'),
                        'penalty_won': statistics['penalty'].get('won'),
                        'penalty_committed': statistics['penalty'].get('commited'),
                        'penalty_scored': statistics['penalty'].get('scored'),
                        'penalty_missed': statistics['penalty'].get('missed'),
                        'penalty_saved': statistics['penalty'].get('saved')
                    })
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON: {e}")
        continue

# Create a DataFrame from the extracted data
extracted_df = pd.DataFrame(extracted_data)

# Display the DataFrame
print(extracted_df.head())


  fixtureID  team_Id team_name  player_id     player_name  minutes  number  \
0   1037952      723   Almeria      46870    Diego Mariño     90.0       1   
1   1037952      723   Almeria      46688  Alejandro Pozo     66.0      17   
2   1037952      723   Almeria       1565  Edgar González     90.0       3   
3   1037952      723   Almeria        340     Srđan Babić     90.0      22   
4   1037952      723   Almeria      47530   Sergio Akieme     90.0      15   

  position rating  captain  ...  dribbles_past  fouls_drawn  fouls_committed  \
0        G    6.9    False  ...            NaN          NaN              NaN   
1        D    6.5    False  ...            NaN          1.0              NaN   
2        D    6.7    False  ...            NaN          1.0              1.0   
3        D    6.9    False  ...            NaN          NaN              2.0   
4        D    7.5    False  ...            1.0          NaN              3.0   

   yellow_cards  red_cards  penalty_won  penalty_c

# ETL: JSON TO EXCEL: PREDICTIONS16DAYS.JSON

In [ ]:
import json
import pandas as pd
from datetime import datetime

# Load the JSON file
json_file_path = '/Users/ardisopi/Documents/DWH/JSON/2024-05-18+19_54+-+predictions16days.json'
with open(json_file_path, 'r') as file:
    data = json.load(file)

# Initialize an empty list to store the extracted information
extracted_data = []

# Function to safely extract data with default fallback
def safe_get(d, keys, default=None):
    for key in keys:
        try:
            d = d[key]
        except (KeyError, IndexError, TypeError):
            return default
    return d

# Function to convert Kelvin to Celsius
def kelvin_to_celsius(kelvin):
    return kelvin - 273.15

# Loop through each city's weather data
for city_data in data:
    city_info = city_data['city']
    city_id = city_info['id']
    city_name = city_info['name']
    coord_lon = city_info['coord']['lon']
    coord_lat = city_info['coord']['lat']
    country = city_info['country']
    population = city_info['population']
    timezone = city_info['timezone']
    
    weather_list = city_data['list']
    
    for weather in weather_list:
        dt = safe_get(weather, ['dt'])
        temp_day_kelvin = safe_get(weather, ['temp', 'day'])
        pressure = safe_get(weather, ['pressure'])
        humidity = safe_get(weather, ['humidity'])
        weather_description = safe_get(weather, ['weather', 0, 'description'])
        wind_speed = safe_get(weather, ['speed'])
        wind_deg = safe_get(weather, ['deg'])
        rain = safe_get(weather, ['rain'], 0)

        # Convert temperatures from Kelvin to Celsius
        temp_day_celsius = kelvin_to_celsius(temp_day_kelvin) if temp_day_kelvin is not None else None
        
        # Convert Unix timestamp to human-readable format
        if dt is not None:
            dt_obj = datetime.utcfromtimestamp(dt)
            date_str = dt_obj.date().isoformat()
            time_str = dt_obj.time().isoformat()
        else:
            date_str = time_str = None
        
        extracted_data.append({
            'city_id': city_id,
            'city_name': city_name,
            'coord_lon': coord_lon,
            'coord_lat': coord_lat,
            'country': country,
            'population': population,
            'timezone': timezone,
            'date': date_str,
            'time': time_str,
            'temp_day': temp_day_celsius,
            'pressure': pressure,
            'humidity': humidity,
            'weather_description': weather_description,
            'wind_speed': wind_speed,
            'wind_deg': wind_deg,
            'rain': rain
        })

# Create a DataFrame from the extracted data
extracted_df = pd.DataFrame(extracted_data)

# Display the DataFrame
print(extracted_df.head())

# Optionally, save the extracted data to a new Excel file
output_path = '/Users/ardisopi/Documents/DWH/extracted_weather_data2.xlsx'
extracted_df.to_excel(output_path, index=False)
print(f"Extracted data saved to {output_path}")


# S3 Bucket - Fetching Data

In [40]:
import boto3
from botocore import UNSIGNED
from botocore.client import Config

# Initialize a session using boto3 with an anonymous client
s3 = boto3.client('s3', config=Config(signature_version=UNSIGNED))

bucket_name = 'dwlprojectbucket'
continuation_token = None

while True:
    if continuation_token:
        response = s3.list_objects_v2(Bucket=bucket_name, ContinuationToken=continuation_token)
    else:
        response = s3.list_objects_v2(Bucket=bucket_name)

    # Check if the response contains 'Contents'
    if 'Contents' in response:
        for obj in response['Contents']:
            print(obj['Key'])
    else:
        print("No more files found in the bucket.")
        break

    # Check if there's more data to retrieve
    if response.get('IsTruncated'):
        continuation_token = response.get('NextContinuationToken')
    else:
        break


2024-05-18 19:42 - lineups.json
2024-05-18 19:42 - predictions4days.json
2024-05-18 19:54 - predictions16days.json
2024-05-18 19:54 - standings.json
2024-05-18 20:24 - fixtures.json
2024-05-18 20:24 - statistics_fixtures_all.json
2024-05-18 20:24 - statistics_players_all.json
2024-05-18 20:42 - lineups.json
2024-05-18 20:42 - predictions4days.json
2024-05-18 21:42 - lineups.json
2024-05-18 21:42 - predictions4days.json
2024-05-18 22:42 - lineups.json
2024-05-18 22:42 - predictions4days.json
2024-05-18 23:42 - lineups.json
2024-05-18 23:42 - predictions4days.json
2024-05-19 00:42 - lineups.json
2024-05-19 00:42 - predictions4days.json
2024-05-19 01:42 - lineups.json
2024-05-19 01:42 - predictions4days.json
2024-05-19 02:42 - lineups.json
2024-05-19 02:42 - predictions4days.json
2024-05-19 03:42 - lineups.json
2024-05-19 03:42 - predictions4days.json
2024-05-19 04:42 - lineups.json
2024-05-19 04:42 - predictions4days.json
2024-05-19 05:42 - lineups.json
2024-05-19 05:42 - predictions4day

2024-06-03 18:43 - predictions4days.json
2024-06-03 18:45 - predictions4days.json
2024-06-03 19:42 - predictions4days.json
2024-06-03 19:43 - predictions4days.json
2024-06-03 19:45 - predictions4days.json
2024-06-03 19:53 - predictions16days.json
2024-06-03 19:53 - standings.json
2024-06-03 20:42 - predictions4days.json
2024-06-03 20:43 - predictions4days.json
2024-06-03 20:45 - predictions4days.json
2024-06-03 21:42 - predictions4days.json
2024-06-03 21:43 - predictions4days.json
2024-06-03 21:45 - predictions4days.json
2024-06-03 22:42 - predictions4days.json
2024-06-03 22:43 - predictions4days.json
2024-06-03 22:45 - predictions4days.json
2024-06-03 23:42 - predictions4days.json
2024-06-03 23:43 - predictions4days.json
2024-06-03 23:45 - predictions4days.json
2024-06-04 00:42 - predictions4days.json
2024-06-04 00:43 - predictions4days.json
2024-06-04 00:45 - predictions4days.json
2024-06-04 01:42 - predictions4days.json
2024-06-04 01:43 - predictions4days.json
2024-06-04 01:45 - pre

# Create the Table sturcture for DIMENSIAL TABLES

In [2]:
# Define PostgreSQL connection parameters
DB_USER = 'rai'
DB_PASSWORD = 'HSLU2024,rai!'
DB_HOST = 'database1.cixcyzlgrsk9.us-east-1.rds.amazonaws.com'
DB_PORT = '5432'
DB_NAME = 'database1'


# Define PostgreSQL connection details  - TESTDATABASE
# DB_HOST = 'database-3.cto9dqorm6ji.us-east-1.rds.amazonaws.com'
# DB_NAME = 'databasetest'
# DB_USER = 'postgres'
# DB_PASSWORD = 'HSLU2024,rai!'
# DB_PORT = 5432

# Establish the connection
conn = psycopg2.connect(
    host=DB_HOST,
    port=DB_PORT,
    database=DB_NAME,
    user=DB_USER,
    password=DB_PASSWORD
)

# Create a cursor object
cursor = conn.cursor()

# SQL commands to create the tables with dimmatches as the last table
create_table_commands = [
    """
    CREATE TABLE IF NOT EXISTS dimvenue (
        venue_id INTEGER PRIMARY KEY,
        venue_name VARCHAR(255),
        venue_city VARCHAR(255)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS dimteams (
        team_id INTEGER PRIMARY KEY,
        team_name VARCHAR(255)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS dimplayer (
        player_id INTEGER PRIMARY KEY,
        player_name VARCHAR(255)
    );
    """,
    """
    CREATE TABLE IF NOT EXISTS dimmatches (
        fixture_id INTEGER PRIMARY KEY,
        referee VARCHAR(255),
        date DATE,
        time TIME,
        timestamp TIMESTAMP,
        status_short VARCHAR(255),
        home_team_id INTEGER,
        home_team_name VARCHAR(255),
        home_winner FLOAT,
        home_goals INTEGER,
        away_team_id INTEGER,
        away_team_name VARCHAR(255),
        away_winner FLOAT,
        away_goals INTEGER,
        venue_id INTEGER
    );
    """
]

# Execute each command to create the tables
for command in create_table_commands:
    try:
        cursor.execute(command)
        print("Table created successfully.")
    except Exception as e:
        print(f"Error creating table: {e}")

# Commit the changes
conn.commit()

# Close the cursor and connection
cursor.close()
conn.close()

print("Database connection closed.")

Table created successfully.
Table created successfully.
Table created successfully.
Table created successfully.
Database connection closed.


# Create the Tables for the FACTS TABLES

In [95]:
import psycopg2
from psycopg2 import sql

# Define PostgreSQL connection parameters
DB_USER = 'rai'
DB_PASSWORD = 'HSLU2024,rai!'
DB_HOST = 'database1.cixcyzlgrsk9.us-east-1.rds.amazonaws.com'
DB_PORT = '5432'
DB_NAME = 'database1'


# PostgreSQL connection details - TESTDATABASE
# DB_HOST = 'database-3.cto9dqorm6ji.us-east-1.rds.amazonaws.com'
# DB_NAME = 'databasetest'
# DB_USER = 'postgres'
# DB_PASSWORD = 'HSLU2024,rai!'
# DB_PORT = 5432

# Connect to the PostgreSQL database
def connect_to_db():
    try:
        connection = psycopg2.connect(
            host=DB_HOST,
            port=DB_PORT,
            database=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD
        )
        print("Successfully connected to the database")
        return connection
    except Exception as e:
        print(f"Error connecting to the database: {str(e)}")
        raise

# SQL statements to create the tables
create_fact_teams_stats_table = """
CREATE TABLE IF NOT EXISTS fact_teams_stats (
    fixture_id INT NOT NULL,
    team_id INT NOT NULL,
    shots_on_goal INT,
    shots_off_goal INT,
    blocked_shots INT,
    shots_insidebox INT,
    shots_outsidebox INT,
    fouls INT,
    corner_kicks INT,
    offsides INT,
    ball_possession FLOAT,
    yellow_cards INT,
    red_cards INT,
    goalkeeper_saves INT,
    total_passes INT,
    passes_accurate INT,
    passes_percentage FLOAT,
    PRIMARY KEY (fixture_id, team_id)
);
"""

create_fact_player_stats_table = """
CREATE TABLE IF NOT EXISTS fact_player_stats (
    fixture_id INT NOT NULL,
    team_id INT NOT NULL,
    player_id INT NOT NULL,
    player_statistics_games_minutes FLOAT,
    player_statistics_games_number INT,
    games_rating FLOAT,
    games_offside INT,
    goals_total INT,
    goals_conceded INT,
    goals_assists INT,
    passes_total INT,
    passes_accuracy FLOAT,
    tackles_total INT,
    duels_total INT,
    fouls_committed INT,
    PRIMARY KEY (fixture_id, team_id, player_id)
);
"""

# Function to create tables
def create_tables():
    try:
        # Connect to the database
        connection = connect_to_db()
        cursor = connection.cursor()

        # Execute the create table statements
        cursor.execute(create_fact_teams_stats_table)
        cursor.execute(create_fact_player_stats_table)

        # Commit the transaction
        connection.commit()

        print("Tables created successfully")

    except Exception as e:
        print(f"Error creating tables: {str(e)}")
    finally:
        if connection:
            cursor.close()
            connection.close()

# Create the tables
create_tables()

# Validate table creation by listing all tables
def list_tables():
    try:
        # Connect to the database
        connection = connect_to_db()
        cursor = connection.cursor()

        # Query to list all tables
        cursor.execute("""
            SELECT table_name
            FROM information_schema.tables
            WHERE table_schema = 'public'
            ORDER BY table_name;
        """)

        # Fetch and print table names
        tables = cursor.fetchall()
        print("Tables in the database:")
        for table in tables:
            print(table[0])

    except Exception as e:
        print(f"Error listing tables: {str(e)}")
    finally:
        if connection:
            cursor.close()
            connection.close()

# List tables to validate creation
list_tables()


Successfully connected to the database
Tables created successfully
Successfully connected to the database
Tables in the database:
dimmatches
dimplayer
dimteams
dimvenue
fact_player_stats
fact_teams_stats


In [87]:
import psycopg2
from psycopg2 import sql

# Define PostgreSQL connection parameters
DB_USER = 'rai'
DB_PASSWORD = 'HSLU2024,rai!'
DB_HOST = 'database1.cixcyzlgrsk9.us-east-1.rds.amazonaws.com'
DB_PORT = '5432'
DB_NAME = 'database1'

# PostgreSQL connection details - TESTDATABASE
# DB_HOST = 'database-3.cto9dqorm6ji.us-east-1.rds.amazonaws.com'
# DB_NAME = 'databasetest'
# DB_USER = 'postgres'
# DB_PASSWORD = 'HSLU2024,rai!'
# DB_PORT = 5432

# Connect to the PostgreSQL database
def connect_to_db():
    try:
        connection = psycopg2.connect(
            host=DB_HOST,
            port=DB_PORT,
            database=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD
        )
        print("Successfully connected to the database")
        return connection
    except Exception as e:
        print(f"Error connecting to the database: {str(e)}")
        raise

# SQL statements to create the weather tables
create_factweatherFC4d_table = """
CREATE TABLE IF NOT EXISTS factweatherFC4d (
    venue_city VARCHAR(255),
    coord_lon FLOAT,
    coord_lat FLOAT,
    country VARCHAR(2),
    population INT,
    timezone INT,
    date DATE,
    time TIME,
    temperature FLOAT,
    pressure INT,
    humidity INT,
    weather_description VARCHAR(255),
    wind_speed FLOAT,
    rain_volume FLOAT
);
"""

create_factweatherFC16d_table = """
CREATE TABLE IF NOT EXISTS factweatherFC16d (
    venue_city VARCHAR(255),
    coord_lon FLOAT,
    coord_lat FLOAT,
    country VARCHAR(2),
    population INT,
    timezone INT,
    date DATE,
    time TIME,
    temperature FLOAT,
    pressure INT,
    humidity INT,
    weather_description VARCHAR(255),
    wind_speed FLOAT,
    rain_volume FLOAT
);
"""

create_factweather_table = """
CREATE TABLE IF NOT EXISTS factweather (
    venue_city VARCHAR(255),
    datetime TIMESTAMP,
    date DATE,
    time TIME,
    temperature FLOAT,
    pressure INT,
    humidity INT,
    wind_speed FLOAT,
    weather_description VARCHAR(255)
);
"""

# Function to create tables
def create_tables():
    try:
        # Connect to the database
        connection = connect_to_db()
        cursor = connection.cursor()

        # Execute the create table statements
        cursor.execute(create_factweatherFC4d_table)
        cursor.execute(create_factweatherFC16d_table)
        cursor.execute(create_factweather_table)

        # Commit the transaction
        connection.commit()

        print("Tables created successfully")

    except Exception as e:
        print(f"Error creating tables: {str(e)}")
    finally:
        if connection:
            cursor.close()
            connection.close()

# Validate table creation by listing all tables
def list_tables():
    try:
        # Connect to the database
        connection = connect_to_db()
        cursor = connection.cursor()

        # Query to list all tables
        cursor.execute("""
            SELECT table_name
            FROM information_schema.tables
            WHERE table_schema = 'public'
            ORDER BY table_name;
        """)

        # Fetch and print table names
        tables = cursor.fetchall()
        print("Tables in the database:")
        for table in tables:
            print(table[0])

    except Exception as e:
        print(f"Error listing tables: {str(e)}")
    finally:
        if connection:
            cursor.close()
            connection.close()

# Create the tables
create_tables()

# List tables to validate creation
list_tables()


Successfully connected to the database
Tables created successfully
Successfully connected to the database
Tables in the database:
dimmatches
dimplayer
dimteams
dimvenue
fact_player_stats
fact_teams_stats
factweather
factweatherfc16d
factweatherfc4d


In [97]:
import psycopg2
from psycopg2 import sql

# Define PostgreSQL connection parameters
DB_USER = 'rai'
DB_PASSWORD = 'HSLU2024,rai!'
DB_HOST = 'database1.cixcyzlgrsk9.us-east-1.rds.amazonaws.com'
DB_PORT = '5432'
DB_NAME = 'database1'

# PostgreSQL connection details - TESTDATABASE
# DB_HOST = 'database-3.cto9dqorm6ji.us-east-1.rds.amazonaws.com'
# DB_NAME = 'databasetest'
# DB_USER = 'postgres'
# DB_PASSWORD = 'HSLU2024,rai!'
# DB_PORT = 5432

# Connect to the PostgreSQL database
def connect_to_db():
    try:
        connection = psycopg2.connect(
            host=DB_HOST,
            port=DB_PORT,
            database=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD
        )
        print("Successfully connected to the database")
        return connection
    except Exception as e:
        print(f"Error connecting to the database: {str(e)}")
        raise

# SQL statements to create the weather tables with primary keys
create_factweather_table = """
CREATE TABLE IF NOT EXISTS factweather (
    venue_city VARCHAR(255),
    temperature FLOAT,
    pressure INT,
    humidity INT,
    wind_speed FLOAT,
    weather_description VARCHAR(255),
    rain_volume FLOAT,
    date DATE,
    time TIME,
    PRIMARY KEY (venue_city, date, time)
);
"""

create_factweatherFC4d_table = """
CREATE TABLE IF NOT EXISTS factweatherFC4d (
    venue_city VARCHAR(255),
    coord_lon FLOAT,
    coord_lat FLOAT,
    country VARCHAR(2),
    population INT,
    timezone INT,
    date DATE,
    time TIME,
    temperature FLOAT,
    pressure INT,
    humidity INT,
    weather_description VARCHAR(255),
    wind_speed FLOAT,
    rain_volume FLOAT,
    PRIMARY KEY (venue_city, date, time)
);
"""

create_factweatherFC16d_table = """
CREATE TABLE IF NOT EXISTS factweatherFC16d (
    venue_city VARCHAR(255),
    coord_lon FLOAT,
    coord_lat FLOAT,
    country VARCHAR(2),
    population INT,
    timezone INT,
    date DATE,
    time TIME,
    temperature FLOAT,
    pressure INT,
    humidity INT,
    weather_description VARCHAR(255),
    wind_speed FLOAT,
    rain_volume FLOAT,
    PRIMARY KEY (venue_city, date, time)
);
"""

# Function to create tables
def create_tables():
    connection = None
    try:
        # Connect to the database
        connection = connect_to_db()
        cursor = connection.cursor()

        # Execute the create table statements in order
        cursor.execute(create_factweather_table)
        cursor.execute(create_factweatherFC4d_table)
        cursor.execute(create_factweatherFC16d_table)

        # Commit the transaction
        connection.commit()

        print("Tables created successfully")

    except Exception as e:
        print(f"Error creating tables: {str(e)}")
    finally:
        if connection:
            cursor.close()
            connection.close()

# Validate table creation by listing all tables
def list_tables():
    connection = None
    try:
        # Connect to the database
        connection = connect_to_db()
        cursor = connection.cursor()

        # Query to list all tables
        cursor.execute("""
            SELECT table_name
            FROM information_schema.tables
            WHERE table_schema = 'public'
            ORDER BY table_name;
        """)

        # Fetch and print table names
        tables = cursor.fetchall()
        print("Tables in the database:")
        for table in tables:
            print(table[0])

    except Exception as e:
        print(f"Error listing tables: {str(e)}")
    finally:
        if connection:
            cursor.close()
            connection.close()

# Create the tables
create_tables()

# List tables to validate creation
list_tables()


Successfully connected to the database
Tables created successfully
Successfully connected to the database
Tables in the database:
dimmatches
dimplayer
dimteams
dimvenue
fact_player_stats
fact_teams_stats
factweather
factweatherfc16d
factweatherfc4d
